In [ ]:
import torch
import torch.nn as nn

In [ ]:
%cd recurrent-ppo/ENV

In [ ]:
from model.trainer import *
import env


In [ ]:
game_name = "MachiKoro"
env.make(game_name)
writer_path = f"/content/runs/{game_name}/"

In [ ]:
env.getActionSize()

In [ ]:
max_eps_length = 200

In [ ]:
config = {
    "num_epochs": 3,
    "embed_dim": 384,
    "max_eps_length":max_eps_length,
    "seq_length":8,
    "hidden_size":128,
    "lr": 2.5e-4,
    "num_game_per_batch":128,
    "gamma":0.99,
    "gae_lambda":0.95,
    "policy_kl_range":0.0008,
    "policy_params": 20,
    "value_clip": 1.0,
    "critic_coef": 1,
    "entropy_coef": 0,
    "max_grad_norm": 0.5,
    "n_mini_batch": 4,
    "rewards": [-1,1], #[lose,win]
    "set_detect_anomaly": True

}


In [ ]:
trainer = Trainer(config=config,env=env,writer_path = writer_path)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/runs/

In [ ]:
trainer.train()